In [1]:
import baggianalysis as ba
import numpy as np

In [2]:
parser = ba.GenericOxDNAParser("ba_topology.dat")
trajectory = ba.FullTrajectory(parser)
trajectory.initialise_from_trajectory_file("trajectory.dat")

In [3]:
def get_eigenvalues(ch_result):
    triangle_coms = np.array(list(map(lambda t: (t.v1 + t.v2 + t.v3) / 3., ch_result.triangles)))
    ch_com = np.average(triangle_coms, axis=0)
    triangle_coms -= ch_com
    
    # https://stackoverflow.com/questions/62153830/how-do-i-efficiently-compute-the-gyration-tensor-in-numpy
    gyr_tensor = np.einsum('im,in->mn', triangle_coms, triangle_coms) / triangle_coms.shape[0]
    ev, _ = np.linalg.eig(gyr_tensor)
    ev = sorted(ev)
    
    return ev


In [4]:
ch = ba.ConvexHull()
trajectory.reset()
system = trajectory.next_frame()
evs = []
while system != None:
    ch.analyse_system(system)
    result = ch.result()
    
    evs.append(get_eigenvalues(result))
    
    system = trajectory.next_frame()

In [5]:
R2 = np.average(evs, axis=0)
strains = evs / R2

J = np.sqrt(np.prod(strains, axis=1))
I = np.sum(strains, axis=1) * J**(-2./3.)

np.savetxt("J_ch.dat", J)
np.savetxt("I_ch.dat", I)

In [6]:
def make_pmf(x, remove_last=None):
    hist, bins = np.histogram(x, bins='auto', density=True)
    bins = bins[:-1] + (bins[1] - bins[0]) / 2.
    result = np.column_stack((bins, hist))
    # get rid of the last points
    if remove_last is not None:
        if len(bins) < 2 * remove_last:
            remove_last = len(bins) / 5
        result = result[result[:,1] > 0][:-remove_last]
    else:
        result = result[result[:,1] > 0]
    result[:,1] = -np.log(result[:,1])

    return result

def fit_J(x, m, D0, b):
    return m * (x - D0)**2. + b

def fit_I(x, m, b):
    return m * x + b

pmf_J = make_pmf(J)
pmf_I = make_pmf(I, 10)

np.savetxt("J_ch_pmf.dat", pmf_J)
np.savetxt("I_ch_pmf.dat", pmf_I)

In [7]:
import scipy.optimize as opt

V_all = 4. * np.pi * np.sqrt(3) * np.prod(np.sqrt(evs), axis=1)
V = np.average(V_all)
print("V average:", V)

x0, y0 = pmf_J[np.argmin(pmf_J[:,1])]
initial_guess = (100., x0, y0)
to_fit = pmf_J[(pmf_J[:,0] > 0.90) & (pmf_J[:,0] < 1.10)]
params, _ = opt.curve_fit(fit_J, to_fit[:,0], to_fit[:,1], p0=initial_guess)
delta, J0, b = params
K = 2. * delta / V
print("J delta:", delta)

to_fit = pmf_I
x_initial = to_fit[0,0]
x_final = to_fit[1,0]
y_initial = to_fit[0,1]
y_final = to_fit[1,1]
initial_slope = (y_final - y_initial) / (x_final - x_initial)
initial_intercept = y_initial - initial_slope * x_initial
initial_guess = (initial_slope, initial_intercept)
params, _ = opt.curve_fit(fit_I, to_fit[:,0], to_fit[:,1], p0=initial_guess)
G = 2. * params[0] / V
print("I slope:", params[0])

Y = 9. * K * G / (3. * K + G)
nu = (3. * K - Y) / (6. * K)

D = 1.5 * (1. - nu**2.) / Y
hertzian_coeff = 1. / (5. * D)

print("K:", K)
print("G:", G)
print("Y:", Y)
print("nu:", nu)
print("D:", D)
print("coeff:", hertzian_coeff)

V average: 8059.610662509122
J delta: 61.182640736104396
I slope: 89.99070304675156
K: 0.015182530099303081
G: 0.022331277977324926
Y: 0.044953720914246195
nu: 0.006519218467747456
D: 33.36623129700734
coeff: 0.005994084205066883
